In [1]:
#Import das bibliotecas
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [2]:
#Instancia e configura um driver para o Chrome
service = Service()
options = webdriver.ChromeOptions()

options.headless = True
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(service=service, options=options)

In [3]:
#Navega para página da loja
url = 'https://www.cskinstore.com/'
driver.get(url)

Etapas manuais:
1 - Logar no site
2 - Aceitar os cookies
3 - Filtrar FACA
4 - Colocar 100 de preco maximo

In [ ]:
teste = 0
valor = 10
while teste == 0:
    time.sleep(5)
    driver.find_elements(By.CLASS_NAME, 'chakra-input.css-4mvnxo')[1].click()
    driver.find_elements(By.CLASS_NAME, 'chakra-input.css-4mvnxo')[1].clear()
    driver.find_elements(By.CLASS_NAME, 'chakra-input.css-4mvnxo')[1].send_keys(valor)
    driver.find_element(By.CLASS_NAME, 'chakra-input.css-nj40x6').click()
    driver.find_element(By.CLASS_NAME, 'chakra-input.css-nj40x6').clear()
    driver.find_element(By.CLASS_NAME, 'chakra-input.css-nj40x6').send_keys('navaja')
    driver.find_element(By.XPATH, '//p[text()="Pesquisar"]').click()
    teste = len(driver.find_elements(By.CLASS_NAME, 'css-l21fsy'))
    valor = valor + 1
    if valor > 580:
        valor = 10

# Adiciona ao carrinho
driver.find_elements(By.CLASS_NAME, 'chakra-button.css-11szynj')[0].click()

# Vai para o carrinho
url = 'https://www.cskinstore.com/cart'
driver.get(url)

# Clica no checkbox
driver.find_elements(By.CLASS_NAME, 'chakra-checkbox__control.css-1n64u0g')[0].click()

# Fecha o pedido
driver.find_element(By.XPATH, '//button[text()="Fechar pedido"]').click()

# Confirma o pedido
driver.find_element(By.XPATH, '//button[text()="Confirmar"]').click()

KeyboardInterrupt: 